**Введите ваше ФИО**

In [ ]:
Мельцов Дмитрий Александрович

# Задание 1

Задача: Создать чат бота для получения информации об исследованиях космоса

Описание: Создайте комплексное приложение командной строки, которое будет использоваться в качестве панели управления исследованиями космоса. Данное приложение будет обращаться к https://api.nasa.gov/ для предоставления пользователям набора информации о космосе, включая:

- Астрономическая картинка дня (APOD): Отображение APOD с пояснениями к нему.
- Фотографии с марсохода: позволяет пользователям выбирать и фильтровать фотографии с марсохода по дате и типу камеры.
- Объекты, сближающиеся с Землей (ОСЗ): Поиск и отображение объектов, сближающихся с Землей, на определенную дату, включая их размеры и потенциальную опасность.
- Данные о космической погоде: Отображают последние данные о космической погоде, включая солнечные вспышки и геомагнитные бури.
Приложение должно позволять пользователям ориентироваться в этих функциях, корректно обрабатывать ошибки и обеспечивать удобство работы.

Требования:
- Пользовательский ввод: Приложение должно предложить пользователю ввести данные, чтобы выбрать, какую функцию он хочет изучить.
- Проверка данных: Убедитесь, что пользовательские данные (например, даты) проверены.
- Обработка ошибок: Корректно обрабатывайте ошибки API и неверные ответы.
- Представление данных: Представляйте данные в четкой и организованной форме.
- Опция выхода: позволяет пользователям выходить из приложения в любое время.

In [ ]:
import requests
from random import randint
from PIL import Image
from IPython.display import display
import io

def watch_apod():
  url = 'https://api.nasa.gov/planetary/apod?api_key=lf9dJGVStKB9qPB1WUKv8qAjINQgYvQLYJOodeEm'
  res = requests.get(url).json()
  picture_url = res['url']
  explanation = res['explanation'].split('.')[0] + '!'
  img_res = requests.get(picture_url)

  img = Image.open(io.BytesIO(img_res.content))
  print(explanation)
  display(img)
  print()

def watch_mars():
  photo_date = input('Введите дату снимка в формате ГГГГ-ММ-ДД: ')
  camera_types = ['fhaz', 'rhaz', 'mast', 'chemcam', 'mahli', 'mardi', 'navcam', 'pancam', 'minites']
  camera = input('Введите тип камеры (аббревиатуру): ')
  if camera.lower() not in camera_types:
    print('Неизвестный тип камеры')
    return 404
  url = f'https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos?earth_date={photo_date}&camera={camera}&api_key=lf9dJGVStKB9qPB1WUKv8qAjINQgYvQLYJOodeEm'

  response = requests.get(url)
  if response.status_code != 200:
    print('Ошибка')
    return response.status_code

  response = requests.get(url).json()
  if len(response['photos']) == 0:
    print('В этот день снимков с этой камеры не было')
    return 'Ошибка'
  rand_pic = randint(0, len(response['photos']) - 1)
  pic_url = response['photos'][rand_pic]['img_src']
  img_res = requests.get(pic_url)
  img = Image.open(io.BytesIO(img_res.content))
  display(img)
  print()

def watch_neo():
  start_date = input('Введите начальную дату для поиска астероида(ГГГГ-ММ-ДД): ')
  end_date = input('Введите конечную дату(ГГГГ-ММ-ДД): ')
  print()
  url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=lf9dJGVStKB9qPB1WUKv8qAjINQgYvQLYJOodeEm'
  response = requests.get(url).json()
  if response.status_code != 200:
    print('Ошибка')
    return response.status_code
  el_count = response['element_count']
  for date in response['near_earth_objects']:
    for asteroid in range(len(date)):
      print("Имя:", response['near_earth_objects'][date][asteroid]['name'])
      if response['near_earth_objects'][date][asteroid]["is_potentially_hazardous_asteroid"] == True:
        print('Потенциально опасен для Земли:')
      else:
        print('Потенциально не опасен для Земли')
      print('Минимальный диаметр:', response['near_earth_objects'][date][asteroid]['estimated_diameter']['meters']['estimated_diameter_min'], 'м')
      print('Максимальный диаметр:', response['near_earth_objects'][date][asteroid]['estimated_diameter']['meters']['estimated_diameter_max'], 'м')
      print()

def watch_donki():
  start_date = input('Введите начальную дату(ГГГГ-ММ-ДД): ')
  end_date = input('Введите конечную дату(ГГГГ-ММ-ДД): ')
  print()
  flr_url = f'https://api.nasa.gov/DONKI/FLR?startDate={start_date}&endDate={end_date}&api_key=lf9dJGVStKB9qPB1WUKv8qAjINQgYvQLYJOodeEm'
  if requests.get(flr_url).status_code != 200:
    print('Ошибка')
    return requests.get(flr_url).status_code
  gst_url = f'https://api.nasa.gov/DONKI/GST?startDate={start_date}&endDate={end_date}&api_key=lf9dJGVStKB9qPB1WUKv8qAjINQgYvQLYJOodeEm'
  if requests.get(gst_url).status_code != 200:
    print('Ошибка')
    return requests.get(gst_url).status_code
  gst_response = requests.get(gst_url).json()
  flr_response = requests.get(flr_url).json()

  print('Информация о солнечных вспышках: ')
  for date in range(len(flr_response)):
    print(flr_response[date]['note'])
  print('Информация о геомагнитных бурях:')
  for date in range(len(gst_response)):
    print(gst_response[date]['allKpIndex'][0]['observedTime'], gst_response[date]['allKpIndex'][0]['kpIndex'])


print('Список команд:')

print('Посмотреть APOD')
print("Посмотреть фото с Марса")
print("Узнать информацию об астероидах")
print('Узнать космическую погоду')
print('Выйти')

while True:
  action = input('Введите команду: ')
  #почему-то после вывода картинки input перестаёт работать
  if action == 'Выйти':
    break
  elif action == 'Посмотреть APOD':
    watch_apod()
  elif action == 'Посмотреть фото с Марса':
    watch_mars()
    print()
    print()
  elif action == 'Узнать информацию об астероидах':
    watch_neo()
  elif action == 'Узнать космическую погоду':
    watch_donki()
  else:
    print('Неизвестная команда')

Output hidden; open in https://colab.research.google.com to view.

# Задание 2

Описание задачи

Цель этой задачи - создать скрипт на Python, который взаимодействует с API Чикагского института искусств (https://api.artic.edu/docs/) для извлечения и отображения произведений искусства. Скрипт должен позволять пользователям просматривать работы по страницам, фильтровать их по имени художника и просматривать подробную информацию о выбранных произведениях искусства. Ниже приведены требования и функциональные возможности, которые необходимо реализовать:

Требования:
Извлекать произведения искусства:

- Создайте функцию, которая извлекает список произведений искусства из API Чикагского института искусств.
Функция должна принимать параметр page для разбивки на страницы и возвращать список произведений искусства вместе с информацией о разбивке на страницы.
Фильтровать произведения искусства:

- Реализуйте функцию, которая фильтрует список произведений искусства на основе имени указанного художника. Функция должна возвращать список работ, которые соответствуют имени художника (без учета регистра).
Отображать подробную информацию об оформлении:

- Напишите функцию, которая отображает названия работ для пользователя и позволяет ему выбрать одну из них, введя соответствующий номер.
После выбора функция должна отображать подробную информацию о выбранном произведении, включая название, исполнителя, дату и носитель.
Разбивка на страницы и взаимодействие с пользователем:

- Создайте основную функцию, которая управляет выборкой произведений и взаимодействием с пользователем.

Разрешите пользователям перемещаться по страницам с произведениями искусства, выполнять фильтрацию по исполнителю или выходить из программы.

Если страниц с произведениями искусства несколько, укажите варианты перехода к следующей странице, предыдущей странице, фильтрации по исполнителю или выхода из программы.

# Задание 3

Задача: Создать программу по управлению портфелем криптовалют

Цель: Создать скрипт на Python, который извлекает цены на криптовалюты в режиме реального времени, позволяет пользователям управлять портфелем криптовалют, вычисляет общую стоимость портфеля, отслеживает изменения цен и предоставляет исторические данные о ценах для анализа.

Требования:
Получение текущих цен на криптовалюты:

Используйте https://docs.coingecko.com/ для получения актуальных цен на список криптовалют.

Управление портфелем:

- Позволяет пользователю создавать портфель криптовалют и управлять им, указывая количество каждой криптовалюты, которой он владеет.
- Расчитывает общую стоимость портфеля в указанной фиатной валюте (например, долларах США).

Отслеживание изменения цен:

- Отображение процентного изменения цены для каждой криптовалюты в портфеле за последние 24 часа.
- Выделите все криптовалюты, стоимость которых значительно увеличилась или снизилась.

Поиск исторических данных о ценах:

- Получение исторических данных о ценах на указанную криптовалюту за последнюю неделю.
- Предоставьте пользователю возможность визуализировать эти данные в простом текстовом формате (например, цены за день).

Взаимодействие с пользователем:

- Реализуйте интерфейс командной строки для ввода данных пользователем.
- Предоставьте опции для получения текущих цен, управления портфелем, просмотра изменений цен или анализа исторических данных.


# Задание 4

Задание 4: Проектное

Вам необходимо самостоятельно найти откртое API предоставляющее информацию в открытом доступе и реализовать собственный проект!


Критерии приемки результата:

- Проект включает в себя не менее 5 возможостей для пользователя
- Проект позволяет использовать все возможности проекта пользователю при помощи взаимодействия через коммандную строку
- Проект работает с открытым API (это значит что при проверке вашей работы преподавателем, преподавателю необходимо просто запустить ячейку с кодом вашего проекта и она будет работать без дополнительных манипуляции)
- Проект должен обязательно включать в себя ряд используемых конструкции:
    - Функции
    - Условные конструкции
    - Ввод/вывод
    - Словари/Списки
- Допускается использование библиотек:
    - requests
    - datetime
    - random

**Здесь добавьте описание вашего проекта**

In [ ]:
#  А здесь код